In [1]:
import cv2
import numpy as np

In [2]:
a = np.array([-1, 2, -4])
np.clip(a, 0, 255)

array([0, 2, 0])

In [3]:
a.astype(np.uint8)

array([255,   2, 252], dtype=uint8)

In [4]:
def get_sobel():
    # 전치를 위해 2차원으로 생성
    der = np.array([[-1, 0, 1]])
    blur = np.array([[1, 2, 1]])

    # dot곱: 순서가 중요함(row벡터가 앞에 오면 의도하지 않은 결과가 나옴)

    # vertical
    x = np.dot(blur.T, der)

    # horizontal
    y = np.dot(der.T, blur)

    return x, y

In [5]:
x, y = get_sobel()
x, y

(array([[-1,  0,  1],
        [-2,  0,  2],
        [-1,  0,  1]]),
 array([[-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]]))

In [6]:
from my_filtering import my_filtering
# 여기에서는 이전 my_filtering 그대로 사용
# clip 과정이 선행되지 않는다면 uint8로 바꿀 때 음수값이 overflow되어 이상해짐

src = cv2.imread('building.jpg', cv2.IMREAD_GRAYSCALE)

dst_x = my_filtering(src, x)
dst_y = my_filtering(src, y)

cv2.imshow('dst_x', dst_x)
cv2.imshow('dst_y', dst_y)
cv2.imshow('original', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
# float32: 1이상의 값은 모두 흰색
# uint8: 255이상의 값은 모두 흰색
src = np.zeros((300, 300))
src[100: 110, :] = 2
src[200: 210, :] = 300

cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
# float32: 0이하의 값은 모두 검은색
# uint8: 0이하의 값은 모두 검은색
src = np.ones((300, 300))
src[100: 110, :] = 0
src[200: 210, :] = -300

cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
# 음수 픽셀에 대해 모두 0으로 인식하는 문제 발견
# my_filtering 수정 - clip 과정, uint8로 바꾸는 과정을 없애야 음수 픽셀이 나타남

x, y = get_sobel()
src = cv2.imread('edge_detection_img.png', cv2.IMREAD_GRAYSCALE)
dst_x = my_filtering(src, x)
dst_y = my_filtering(src, y)

cv2.imshow('dst_x', dst_x)
cv2.imshow('dst_y', dst_y)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
x, y = get_sobel()
src = cv2.imread('edge_detection_img.png', cv2.IMREAD_GRAYSCALE)
dst_x = my_filtering(src, x)
dst_y = my_filtering(src, y)

# 음수 픽셀에 절대값을 취하여 양수로
dst_x = np.abs(dst_x)
dst_y = np.abs(dst_y)

dst = dst_x + dst_y

cv2.imshow('dst_x', dst_x)
cv2.imshow('dst_y', dst_y)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
x, y = get_sobel()
src = cv2.imread('edge_detection_img.png', cv2.IMREAD_GRAYSCALE)
dst_x = my_filtering(src, x)
dst_y = my_filtering(src, y)

# normalization 적용 [0, 1]
dst_x_norm = (dst_x - dst_x.min()) / (dst_x - dst_x.min()).max()
dst_y_norm = (dst_y - dst_y.min()) / (dst_y - dst_y.min()).max()

cv2.imshow('dst_x_norm', dst_x_norm)
cv2.imshow('dst_y_norm', dst_y_norm)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
x, y = get_sobel()
src = cv2.imread('building.jpg', cv2.IMREAD_GRAYSCALE)
dst_x = my_filtering(src, x)
dst_y = my_filtering(src, y)

# 음수 픽셀에 절대값을 취하여 양수로
dst_x = np.abs(dst_x)
dst_y = np.abs(dst_y)

dst = dst_x + dst_y

cv2.imshow('dst_x', dst_x)
cv2.imshow('dst_y', dst_y)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [16]:
import cv2
import numpy as np

# library add
import os, sys
from my_filtering import my_filtering


def get_DoG_filter(fsize, sigma=1):
    ###################################################
    # TODO                                            #
    # DoG mask 완성                                    #
    ###################################################
    y, x = np.mgrid[-(fsize//2):(fsize//2)+1, -(fsize//2):(fsize//2)+1]

    DoG_x = (-x / sigma ** 2) * np.exp(-(x ** 2 + y ** 2) / (2 * sigma**2))
    DoG_y = (-y / sigma ** 2) * np.exp(-(x ** 2 + y ** 2) / (2 * sigma**2))

    return DoG_x, DoG_y

In [24]:
src = cv2.imread('Lena.png', cv2.IMREAD_GRAYSCALE)
DoG_x, DoG_y = get_DoG_filter(fsize=3, sigma=1)

###################################################
# TODO                                            #
# DoG mask sigma값 조절해서 mask 만들기              #
###################################################
# DoG_x, DoG_y filter 확인
x, y = get_DoG_filter(fsize=256, sigma=100) 
x = ((x - x.min()) / (x - x.min()).max() * 255).astype(np.uint8)
y = ((y - y.min()) / (y - y.min()).max() * 255).astype(np.uint8)

dst_x = np.abs(my_filtering(src, DoG_x))
dst_y = np.abs(my_filtering(src, DoG_y))

###################################################
# TODO                                            #
# dst_x, dst_y 를 사용하여 magnitude 계산            #
###################################################
dst = np.sqrt((dst_x ** 2 + dst_y ** 2))

"""
한글이나 영어로 작성하기
ID, name(학번 이름): 2022012340, Hon Gil Dong    
department(학과): computer science
"""

cv2.imshow('DoG_x filter - 2022012340 Hong Gil Dong', x)
cv2.imshow('DoG_y filter - 2022012340 Hong Gil Dong', y)
cv2.imshow('dst_x - 2022012340 Hong Gil Dong', dst_x/255)
cv2.imshow('dst_y - 2022012340 Hong Gil Dong', dst_y/255)
cv2.imshow('dst - 2022012340 Hong Gil Dong', dst/255)
cv2.waitKey()
cv2.destroyAllWindows()